In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 16.2 MB/s eta 0:00:00a 0:00:01


In [3]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [4]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
import wandb

wandb.login(key="7018987bed28331a412045428cc3a120465b9504")


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
dataset_path = '/kaggle/input/pedestriandataset'
images_path = f"{dataset_path}/images"
labels_path = f"{dataset_path}/etiquetas"


In [8]:
working_labels_path = '/kaggle/working/labels'
if not os.path.exists(working_labels_path):
    shutil.copytree(labels_path, working_labels_path)

In [9]:

# List images and corresponding labels
image_files = sorted([f for f in os.listdir(images_path) if f.endswith('.jpg')])
label_files = sorted([f.replace('.jpg', '.txt') for f in image_files])

# Split into train and validation
train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=108)
train_labels = [f.replace('.jpg', '.txt') for f in train_images]
val_labels = [f.replace('.jpg', '.txt') for f in val_images]

# Create directories for train and val data
os.makedirs('/kaggle/working/images/train', exist_ok=True)
os.makedirs('/kaggle/working/images/val', exist_ok=True)
os.makedirs('/kaggle/working/labels/train', exist_ok=True)
os.makedirs('/kaggle/working/labels/val', exist_ok=True)

# Move files to train and val folders
for img, lbl in zip(train_images, train_labels):
    shutil.copy2(f"{images_path}/{img}", f"/kaggle/working/images/train/{img}")
    shutil.copy2(f"{working_labels_path}/{lbl}", f"/kaggle/working/labels/train/{lbl}")

for img, lbl in zip(val_images, val_labels):
    shutil.copy2(f"{images_path}/{img}", f"/kaggle/working/images/val/{img}")
    shutil.copy2(f"{working_labels_path}/{lbl}", f"/kaggle/working/labels/val/{lbl}")


In [10]:
model = YOLO('yolov8x.pt') 

100%|██████████| 131M/131M [00:00<00:00, 350MB/s] 


In [12]:
import yaml

data_yaml = {
    'path': '/kaggle/working',
    'train': 'images/train',
    'val': 'images/val',
    'nc': 4,
    'names': ['MMNAP', 'MMAP', 'MNAP', 'PCP']
}

with open("/kaggle/working/dataset.yaml", "w") as f:
    yaml.dump(data_yaml, f)


In [13]:


with open('/kaggle/working/dataset.yaml', 'r') as file:
    try:
        yaml_content = yaml.safe_load(file)
        
        # Now you can check the content
        print(yaml_content)

    except yaml.YAMLError as exc:
        print(exc)

{'names': ['MMNAP', 'MMAP', 'MNAP', 'PCP'], 'nc': 4, 'path': '/kaggle/working', 'train': 'images/train', 'val': 'images/val'}


In [ ]:
results = model.train(
    data='/kaggle/working/dataset.yaml',
    imgsz=640,
    epochs=100,  # Reduced epochs to 100 for testing
    batch=8,
    name='pedestrian_violation'
)

Ultralytics 8.3.27 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_violation, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]
2024-11-02 06:02:06,639	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-02 06:02:07,226	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 62.0MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     34/100      7.67G      1.583     0.9939      1.429          7        640: 100%|██████████| 633/633 [06:21<00:00,  1.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 80/80 [00:22<00:00,  3.56it/s]

                   all       1265       2209      0.822      0.767      0.829       0.41



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     35/100      7.66G      1.554     0.9712      1.405         28        640:  76%|███████▋  | 484/633 [04:51<01:29,  1.66it/s]